### RiskCalc examples.
Make sure that the working directory for this ipynb notebook is ./risktables/risktables

___
The class ```RiskCalcs``` performs basic portfolio risk calculations.  It returns a dictionary, whose items are either instances of Pandas ```DataFrame``` or floating point values representing portfolio aggregate risk measurements (VaR, Greeks, etc).
___

In [1]:
# include both the project package, and the project folder in sys.path
import sys,os
if  not './' in sys.path:
    sys.path.append(os.path.abspath('./'))
if  not '../' in sys.path:
    sys.path.append(os.path.abspath('../'))

from risktables import risk_tables
import pandas as pd
from IPython import display
import datetime

#### Create an instance of RiskCalcs

In [2]:
# df_postgres_info = pd.read_csv('postgres_info.csv')
# display.display(df_postgres_info)
# prices_db_info = df_postgres_info.loc[df_postgres_info.config_name=='dashrisk_local']
rt = risk_tables.RiskCalcs(use_postgres=False)

#### Create an example portfolio consisting of hypothetical options on the S&P 500 Sector Spdr's

In [3]:
next_year = (datetime.datetime.now() + datetime.timedelta(weeks=52)).year
next_year = str(next_year)
df_spdr_options =  pd.read_csv('spdr_stocks.csv')
df_spdr_options.symbol = df_spdr_options.symbol.apply(lambda s: s.replace('2021',next_year))
display.display(df_spdr_options)


,symbol,position
0,XLB_20201222_58_c,172
1,XLC_20201222_49_p,204
2,XLE_20201222_67_c,149
3,XLF_20201222_27_p,370
4,XLI_20201222_78_c,128
5,XLK_20201222_78_p,128
6,XLP_20201222_57_c,175
7,XLU_20201222_57_p,175
8,XLV_20201222_86_c,116
9,XLY_20201222_119_p,84


#### Run risk calculations
*Market data must be fetched for each underlying, which causes the cell below to take about 10 seconds to run*

In [4]:
risk_dictionary = rt.calculate(df_spdr_options)

2019-07-30 13:06:43,662 - root - INFO - Start computing VaR 2019-07-30 13:06:43.662726


entering update_delta_div
End computing VaR 2019-07-30 13:06:52.784067
leaving update_memory


#### List the names of the outputs

In [5]:
for k in risk_dictionary.keys():
    print(k)

yyyymmddhhmmssmmmmmm
df_std
df_atm_price
df_high_low
df_atm_info
df_risk_all
df_risk_by_underlying
df_var
port_var
sp_dollar_equiv
delta
gamma
vega
theta
df_hedge_ratios
df_corr
df_corr_price


#### Now show the actual values of each key

In [6]:
for k in risk_dictionary.keys():
    print(f'********************* {k} **************************')
    dict_item = risk_dictionary[k]
    object_to_print = dict_item
    if type(dict_item)==dict:
        if k[:2] == 'df':
            object_to_print = risk_tables.make_df(risk_dictionary[k])
    display.display(object_to_print)


********************* yyyymmddhhmmssmmmmmm **************************


'20190730130652784067'

********************* df_std **************************


,stdev,underlying
0,0.103427,XLV
1,0.121804,XLY
2,0.166557,XLK
3,0.105998,XLP
4,0.173353,XLE
5,0.124197,XLU
6,0.130894,XLF
7,0.169331,XLC
8,0.151587,XLB
9,0.138847,XLI


********************* df_atm_price **************************


...

""


********************* df_corr **************************


,*underlying,XLB,XLC,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY
XLB,XLB,1.000000,0.193207,0.552811,0.709393,0.782189,0.495035,0.531099,0.049177,0.459971,0.568985
XLC,XLC,0.193207,1.000000,0.328135,0.436789,0.395205,0.715893,0.279818,-0.021327,0.436573,0.733941
XLE,XLE,0.552811,0.328135,1.000000,0.582450,0.623944,0.551978,0.157243,-0.249161,0.245868,0.554188
XLF,XLF,0.709393,0.436789,0.582450,1.000000,0.737178,0.623703,0.368372,0.050491,0.513261,0.725719
XLI,XLI,0.782189,0.395205,0.623944,0.737178,1.000000,0.660098,0.360786,-0.059828,0.422315,0.731479
XLK,XLK,0.495035,0.715893,0.551978,0.623703,0.660098,1.000000,0.319235,-0.085544,0.543968,0.857389
XLP,XLP,0.531099,0.279818,0.157243,0.368372,0.360786,0.319235,1.000000,0.506932,0.631848,0.434606
XLU,XLU,0.049177,-0.021327,-0.249161,0.050491,-0.059828,-0.085544,0.506932,1.000000,0.519114,-0.030187
XLV,XLV,0.459971,0.436573,0.245868,0.513261,0.422315,0.543968,0.631848,0.519114,1.000000,0.550605
XLY,XLY,0.568985,0.733941,0.554188,0.725719,0.731479,0.857389,0.434606,-0.030187,0.550605,1.000000


********************* df_corr_price **************************


,*underlying,XLB,XLC,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY
XLB,XLB,1.000000,0.662157,0.456044,0.854112,0.911985,0.890177,0.895907,0.797522,0.904428,0.876175
XLC,XLC,0.662157,1.000000,0.738232,0.877544,0.848107,0.895885,0.839357,0.627586,0.702922,0.910332
XLE,XLE,0.456044,0.738232,1.000000,0.621440,0.704254,0.645540,0.560478,0.397270,0.523563,0.682871
XLF,XLF,0.854112,0.877544,0.621440,1.000000,0.931462,0.959136,0.921466,0.704459,0.777315,0.943275
XLI,XLI,0.911985,0.848107,0.704254,0.931462,1.000000,0.962485,0.917572,0.733133,0.845756,0.949155
XLK,XLK,0.890177,0.895885,0.645540,0.959136,0.962485,1.000000,0.951817,0.734154,0.828622,0.977896
XLP,XLP,0.895907,0.839357,0.560478,0.921466,0.917572,0.951817,1.000000,0.831020,0.849384,0.953150
XLU,XLU,0.797522,0.627586,0.397270,0.704459,0.733133,0.734154,0.831020,1.000000,0.906522,0.780451
XLV,XLV,0.904428,0.702922,0.523563,0.777315,0.845756,0.828622,0.849384,0.906522,1.000000,0.847518
XLY,XLY,0.876175,0.910332,0.682871,0.943275,0.949155,0.977896,0.953150,0.780451,0.847518,1.000000
